Group of Mingyue Zhang (N16336920) and Wei-Han Huang 

In [ ]:
import yfinance as yf, pandas as pd, shutil, os
from get_all_tickers import get_tickers as gt

In [2]:
usa = ['/SP','NDAQ','^NYA']
south = {'Australia':'^AXJO','South Africa':'EZA','New Zealand':'^NZ50'}
north = {'USA':'/SP','UK':'EWU','Sweden':'EWD','Canada':'EWC','Germany':'EWG','Japan':'^N225'}

In [16]:
df = yf.download(usa+list(north.values())+list(south.values()),start='2006-01-01')['Close'].dropna()

[*********************100%***********************]  11 of 11 completed


In [17]:
df.describe()

,/SP,EWC,EWD,EWG,EWU,EZA,NDAQ,^AXJO,^N225,^NYA,^NZ50
count,3345.000000,3345.000000,3345.000000,3345.000000,3345.000000,3345.000000,3345.000000,3345.000000,3345.000000,3345.000000,3345.000000
mean,23.315439,27.172353,29.592311,26.322975,35.359366,57.232641,50.787196,5307.351781,15787.734368,9781.987902,5670.211996
std,7.534018,3.453034,5.110573,4.531160,6.473342,9.982282,29.954911,779.058561,4858.334440,2176.635517,2711.328772
min,9.600000,13.790000,11.510000,12.730000,18.000000,24.480000,15.760000,3145.500000,7054.979980,4226.310059,2417.949951
25%,17.540001,25.430000,26.740000,22.959999,31.600000,51.860001,26.170000,4781.200195,10512.690430,8113.240234,3426.780029
50%,21.459999,27.660000,30.330000,26.670000,34.400002,58.189999,39.330002,5346.899902,16123.269531,9844.849609,4610.310059
75%,26.360001,29.120001,33.410000,29.760000,38.779999,64.800003,70.820000,5876.200195,19869.849609,11371.839844,7412.120117
max,46.509998,36.419998,40.060001,36.500000,54.439999,76.680000,137.279999,7162.500000,26817.939453,14516.730469,12919.250000


###  Daylight Calculator

In [18]:
from datetime import datetime
import sun
import pandas as pd

def daylight(time,lat,long):
    s = sun.sun(lat=lat, long=long)
    daylight = []
    for timestamp in time:
        daylight.append(min(s.daylight(when=timestamp)-12,0))
    return daylight


#coordinates of the exchanges
coor = {'USA':[40.7128, -74.0060],'Australia':[-33.8688,151.2093],'South Africa':[-26.2041, 28.0473],'New Zealand':[-41.2749,174.7745],
       'UK':[51.5074,-0.1278],'Sweden':[59.3293,18.0686],'Canada':[43.6532,-79.3832],'Germany':[50.1109,8.6821],'Japan':[35.6804,139.769]}

## Dummy Variables

In [19]:
def dummy_fall_and_Monday(time,hemisphere='north'):
    if hemisphere == 'north':
        month = 9
    else:
        month = 3
    date = pd.DataFrame({'month':time.month,'day':time.day})
    fall = pd.DataFrame([0]*len(date))
    Monday = pd.DataFrame([0]*len(date))
    fall[((date.month==month) & (date.day>=21)) |((date.month==month+1) | 
                                                  (date.month==month+2))|((date.month==month+3) & (date.day<=20))]=1
    Monday[time.weekday==0] = 1

    return list(fall[0]), list(Monday[0])

In [20]:
import numpy as np

#function to find the nearest date to the given date
def nearest_ind(items, pivot):
    time_diff = [(date - pivot).days for date in items]   
    return time_diff.index(min([i for i in time_diff if i >=0]))

def dummy_tax(time, firstDay):
    #tax = pd.DataFrame([0]*len(time))
    tax = np.array([0]*len(time))
    for i in range(time[0].year,time[-1].year+1):
        firstDate = datetime(year=i,month=firstDay[0],day=firstDay[1])
        idx = nearest_ind(time,firstDate)
        if (time[idx]-firstDate).days>10:
            continue
        else:
            tax[max(idx-1,0):idx+5]= 1
    return list(tax)

In [21]:
tax={'USA':[1,1],'Australia':[7,1],'South Africa':[3,1],'New Zealand':[4,1],
       'UK':[4,6],'Sweden':[1,1],'Canada':[1,1],'Germany':[1,1],'Japan':[1,1]}

## Add SAD and dummy variables to dataset

In [22]:
# fall and Monday
fall_north, Monday = dummy_fall_and_Monday(df.index)
fall_south, Monday = dummy_fall_and_Monday(df.index,'south')
df['fall_south'] = fall_south
df['fall_north'] = fall_north
df['Monday'] = Monday

In [23]:
for city, coordinate in coor.items():
    df['SAD_'+city] = daylight(df.index,lat=coor[city][0],long=coor[city][1])

In [24]:
df.head()

,/SP,EWC,EWD,EWG,EWU,EZA,NDAQ,^AXJO,^N225,^NYA,...,Monday,SAD_USA,SAD_Australia,SAD_South Africa,SAD_New Zealand,SAD_UK,SAD_Sweden,SAD_Canada,SAD_Germany,SAD_Japan
Date,,,,,,,,,,,,,,,,,,,,,
2006-01-04,9.745,22.719999,23.719999,21.280001,38.759998,51.869999,36.939999,4820.299805,16361.540039,7962.939941,...,0,-2.665833,0.0,0.0,0.0,-4.041944,-5.710833,-2.980833,-3.822500,-2.194444
2006-01-05,9.745,22.379999,23.690001,21.200001,38.540001,51.400002,36.950001,4813.700195,16425.369141,7944.060059,...,0,-2.651944,0.0,0.0,0.0,-4.019444,-5.675556,-2.965000,-3.801667,-2.182778
2006-01-06,9.695,22.600000,23.950001,21.400000,39.180000,53.224998,39.840000,4791.100098,16428.210938,8031.660156,...,0,-2.636667,0.0,0.0,0.0,-3.995278,-5.638056,-2.948056,-3.779167,-2.170556
2006-01-10,9.750,22.540001,23.650000,21.139999,38.720001,53.410000,38.270000,4816.700195,16124.349609,8030.589844,...,0,-2.566389,0.0,0.0,0.0,-3.883611,-5.463611,-2.868611,-3.674444,-2.112778
2006-01-11,9.705,22.670000,23.940001,21.340000,39.000000,54.564999,38.889999,4838.100098,16363.589844,8063.509766,...,0,-2.546111,0.0,0.0,0.0,-3.851944,-5.414167,-2.846111,-3.644444,-2.096389


In [25]:
for country, firstday in tax.items():
    df['tax_'+country] = dummy_tax(df.index,tax[country])

In [26]:
df

,/SP,EWC,EWD,EWG,EWU,EZA,NDAQ,^AXJO,^N225,^NYA,...,SAD_Japan,tax_USA,tax_Australia,tax_South Africa,tax_New Zealand,tax_UK,tax_Sweden,tax_Canada,tax_Germany,tax_Japan
Date,,,,,,,,,,,,,,,,,,,,,
2006-01-04,9.745000,22.719999,23.719999,21.280001,38.759998,51.869999,36.939999,4820.299805,16361.540039,7962.939941,...,-2.194444,1,0,0,0,0,1,1,1,1
2006-01-05,9.745000,22.379999,23.690001,21.200001,38.540001,51.400002,36.950001,4813.700195,16425.369141,7944.060059,...,-2.182778,1,0,0,0,0,1,1,1,1
2006-01-06,9.695000,22.600000,23.950001,21.400000,39.180000,53.224998,39.840000,4791.100098,16428.210938,8031.660156,...,-2.170556,1,0,0,0,0,1,1,1,1
2006-01-10,9.750000,22.540001,23.650000,21.139999,38.720001,53.410000,38.270000,4816.700195,16124.349609,8030.589844,...,-2.112778,1,0,0,0,0,1,1,1,1
2006-01-11,9.705000,22.670000,23.940001,21.340000,39.000000,54.564999,38.889999,4838.100098,16363.589844,8063.509766,...,-2.096389,1,0,0,0,0,1,1,1,1
2006-01-12,9.600000,22.510000,23.610001,21.139999,38.759998,53.500000,38.299999,4836.899902,16445.189453,8008.089844,...,-2.079444,0,0,0,0,0,0,0,0,0
2006-01-13,9.735000,22.629999,23.629999,21.170000,39.119999,54.349998,39.009998,4836.700195,16454.949219,8025.939941,...,-2.061389,0,0,0,0,0,0,0,0,0
2006-01-17,9.760000,22.760000,23.500000,21.059999,38.740002,52.915001,38.240002,4866.100098,15805.950195,7992.600098,...,-1.982222,0,0,0,0,0,0,0,0,0
2006-01-18,9.675000,22.340000,23.090000,20.860001,38.380001,52.529999,36.900002,4786.899902,15341.179688,7953.009766,...,-1.960556,0,0,0,0,0,0,0,0,0


In [27]:
df.to_csv('data.csv')

## Import Weather Data

In [28]:
from scipy.interpolate import interp1d
from sklearn.impute import SimpleImputer


for country in tax.keys():
    weather = pd.read_csv('../Project_ts/Weather/'+country+'.csv')
    weather = weather[['DATE','PRCP','TAVG']]
    weather.DATE = pd.to_datetime(weather.DATE)
    weather.set_index('DATE',inplace=True)
    
    #interpolation
    weather['TAVG'] = weather['TAVG'].interpolate()

    weather.fillna(0,inplace=True)
    
    weather.rename(columns={'PRCP': 'PRCP_'+country, 'TAVG': 'TAVG_'+country},inplace=True)
    
    df = df.merge(weather,left_index=True,right_index=True,how='left')
#df.dropna(inplace=True)

In [30]:
df.to_csv('data.csv')

In [32]:
df.dropna(inplace=True)

In [50]:
df = df.replace([np.inf, -np.inf,np.nan,'#DIV/0!'],0)

In [51]:
df.astype(float)

,/SP,EWC,EWD,EWG,EWU,EZA,NDAQ,^AXJO,^N225,^NYA,...,PRCP_UK,TAVG_UK,PRCP_Sweden,TAVG_Sweden,PRCP_Canada,TAVG_Canada,PRCP_Germany,TAVG_Germany,PRCP_Japan,TAVG_Japan
2006-01-04,9.745000,22.719999,23.719999,21.280001,38.759998,51.869999,36.939999,4820.299805,16361.540039,7962.939941,...,1.0,4.8,0.3,-7.80,9.4,2.75,0.0,-0.3,0.0,4.4
2006-01-05,9.745000,22.379999,23.690001,21.200001,38.540001,51.400002,36.950001,4813.700195,16425.369141,7944.060059,...,0.0,2.4,0.0,-3.05,0.2,1.75,0.0,1.8,0.0,2.8
2006-01-06,9.695000,22.600000,23.950001,21.400000,39.180000,53.224998,39.840000,4791.100098,16428.210938,8031.660156,...,0.0,2.5,0.0,-0.25,0.6,-7.00,0.0,2.8,0.0,2.1
2006-01-10,9.750000,22.540001,23.650000,21.139999,38.720001,53.410000,38.270000,4816.700195,16124.349609,8030.589844,...,0.0,6.2,2.1,-3.00,1.0,2.00,0.0,-2.0,0.0,4.8
2006-01-11,9.705000,22.670000,23.940001,21.340000,39.000000,54.564999,38.889999,4838.100098,16363.589844,8063.509766,...,0.8,7.4,0.0,1.55,0.0,3.25,2.3,-2.8,0.0,6.3
2006-01-12,9.600000,22.510000,23.610001,21.139999,38.759998,53.500000,38.299999,4836.899902,16445.189453,8008.089844,...,0.8,2.6,0.0,3.80,0.0,6.00,0.0,0.2,0.0,6.1
2006-01-13,9.735000,22.629999,23.629999,21.170000,39.119999,54.349998,39.009998,4836.700195,16454.949219,8025.939941,...,0.3,6.6,0.0,1.25,2.5,4.75,0.0,-0.8,0.0,4.7
2006-01-17,9.760000,22.760000,23.500000,21.059999,38.740002,52.915001,38.240002,4866.100098,15805.950195,7992.600098,...,4.3,7.3,0.2,0.30,26.8,-1.50,10.0,-0.4,0.0,6.7
2006-01-18,9.675000,22.340000,23.090000,20.860001,38.380001,52.529999,36.900002,4786.899902,15341.179688,7953.009766,...,0.5,9.3,0.5,-2.95,2.4,2.00,0.2,3.3,0.0,5.3
2006-01-19,9.690000,22.719999,23.170000,20.920000,38.619999,55.474998,38.480000,4840.899902,15696.280273,8018.220215,...,0.3,9.4,0.5,-7.90,0.0,1.50,0.1,0.6,0.0,3.5


## Analysis

In [104]:
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
results={}

for country in tax.keys():
    if country in north:
        index = north[country]
        fall = 'fall_north'
    else:
        index = south[country]
        fall = 'fall_south'
    y = np.log(df[index]).diff()
   
    X = df[[fall,'Monday','PRCP_'+country,'TAVG_'+country,'SAD_'+city,'tax_'+country]]
    X['return_lag1'] = y.shift(1)
    X['return_lag2'] = y.shift(2)
    X = X[5:]
    y = y[5:]
    X = sm.add_constant(X)
    #X = df.replace([np.inf, -np.inf,np.nan],0)
    #X[['return_lag1','return_lag2','PRCP_'+country,'TAVG_'+country,'SAD_'+city,'tax_'+country]] = scaler.fit_transform(X[['return_lag2','return_lag1','PRCP_'+country,'TAVG_'+country,'SAD_'+city,'tax_'+country]])
    y = y.reset_index(drop = True)
    X = X.reset_index(drop = True)
    model = sm.OLS(y,X.astype(float))
    results[country] = model.fit()
    

C:\Users\zmy_b\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\Users\zmy_b\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
C:\Users\zmy_b\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.

In [107]:
for country in tax.keys():
    
    print(results[country].summary().as_latex())

\begin{center}
\begin{tabular}{lclc}
\toprule
\textbf{Dep. Variable:}    &       /SP        & \textbf{  R-squared:         } &     0.005   \\
\textbf{Model:}            &       OLS        & \textbf{  Adj. R-squared:    } &     0.002   \\
\textbf{Method:}           &  Least Squares   & \textbf{  F-statistic:       } &     1.934   \\
\textbf{Date:}             & Wed, 23 Dec 2020 & \textbf{  Prob (F-statistic):} &   0.0509    \\
\textbf{Time:}             &     00:43:18     & \textbf{  Log-Likelihood:    } &    7319.1   \\
\textbf{No. Observations:} &        3204      & \textbf{  AIC:               } & -1.462e+04  \\
\textbf{Df Residuals:}     &        3195      & \textbf{  BIC:               } & -1.457e+04  \\
\textbf{Df Model:}         &           8      & \textbf{                     } &             \\
\bottomrule
\end{tabular}
\begin{tabular}{lcccccc}
                      & \textbf{coef} & \textbf{std err} & \textbf{t} & \textbf{P$> |$t$|$} & \textbf{[0.025} & \textbf{0.975]}  \\
\mi

In [99]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X[['SAD_Japan','TAVG_Japan']] = scaler.fit_transform(X[['SAD_Japan','TAVG_Japan']])

In [111]:
tax.keys()

dict_keys(['USA', 'Australia', 'South Africa', 'New Zealand', 'UK', 'Sweden', 'Canada', 'Germany', 'Japan'])

In [118]:
print(results['Japan'].summary().as_latex())

\begin{center}
\begin{tabular}{lclc}
\toprule
\textbf{Dep. Variable:}    &      ^N225       & \textbf{  R-squared:         } &     0.004   \\
\textbf{Model:}            &       OLS        & \textbf{  Adj. R-squared:    } &     0.001   \\
\textbf{Method:}           &  Least Squares   & \textbf{  F-statistic:       } &     1.416   \\
\textbf{Date:}             & Wed, 23 Dec 2020 & \textbf{  Prob (F-statistic):} &    0.184    \\
\textbf{Time:}             &     01:36:32     & \textbf{  Log-Likelihood:    } &    8682.7   \\
\textbf{No. Observations:} &        3204      & \textbf{  AIC:               } & -1.735e+04  \\
\textbf{Df Residuals:}     &        3195      & \textbf{  BIC:               } & -1.729e+04  \\
\textbf{Df Model:}         &           8      & \textbf{                     } &             \\
\bottomrule
\end{tabular}
\begin{tabular}{lcccccc}
                      & \textbf{coef} & \textbf{std err} & \textbf{t} & \textbf{P$> |$t$|$} & \textbf{[0.025} & \textbf{0.975]}  \\
\mi